# Egunean Behin Visual Question Answering Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install transformers==4.15.0 timm==0.4.12 fairscale==0.4.4
!git clone https://github.com/salesforce/BLIP
%cd BLIP

     |████████████████████████████████| 3.4 MB 5.4 MB/s 
     |████████████████████████████████| 376 kB 44.6 MB/s 
     |████████████████████████████████| 235 kB 48.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 30.8 MB/s 
     |████████████████████████████████| 3.3 MB 24.4 MB/s 
     |████████████████████████████████| 67 kB 2.2 MB/s 
     |████████████████████████████████| 596 kB 35.5 MB/s 
  Created wheel for fairscale: filename=fairscale-0.4.4-py3-none-any.whl size=292864 sha256=49a80feb17e8afec402ff00345151e504dd8592cb43537acda63aade2adc6101
  Stored in directory: /root/.cache/pip/wheels/7e/01/9d/ea7ac6a01eb4044323d028f92fe2ac26c36157ad01b2b907c1
Successfully built fairscale
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalle

In [42]:
!pip3 install cairosvg

     |████████████████████████████████| 45 kB 2.9 MB/s 
     |████████████████████████████████| 88 kB 6.7 MB/s 
  Created wheel for cairocffi: filename=cairocffi-1.3.0-py3-none-any.whl size=89668 sha256=4baf3dce865cffbbf7f7847ba3f392eda66a8dc8212c6272de1c54954b08ffad
  Stored in directory: /root/.cache/pip/wheels/4e/ca/e1/5c8a9692a27f639a07c949044bec943f26c81cd53d3805319f
Successfully built cairocffi


## Create Dataset

### Figures

In [3]:
path_figures = "/content/drive/MyDrive/LAP/Subjects/DL/project/egunean-behin-vqa/data/figures/"

In [24]:
%cd $path_figures

/content/drive/MyDrive/LAP/Subjects/DL/project/egunean-behin-vqa/data/figures


In [25]:
!python create_images.py --n 99

Images: 100% 99/99 [00:01<00:00, 51.69it/s]


In [26]:
!python create_questions.py

Questions: 100% 100/100 [00:00<00:00, 3577.48it/s]


### Cubes

In [4]:
path_cubes = "/content/drive/MyDrive/LAP/Subjects/DL/project/egunean-behin-vqa/data/cubes/"

In [28]:
%cd $path_cubes

/content/drive/MyDrive/LAP/Subjects/DL/project/egunean-behin-vqa/data/cubes


In [31]:
!python create_images.py --n 99

Images: 100% 99/99 [00:29<00:00,  3.41it/s]


In [38]:
!python create_questions.py

Questions: 100% 100/100 [00:00<00:00, 3503.63it/s]


### Maze

In [5]:
path_maze = "/content/drive/MyDrive/LAP/Subjects/DL/project/egunean-behin-vqa/data/maze/"

In [40]:
%cd $path_maze

/content/drive/MyDrive/LAP/Subjects/DL/project/egunean-behin-vqa/data/maze


In [44]:
!python create_images.py --n 100

Images: 100% 100/100 [00:58<00:00,  1.71it/s]


In [45]:
!python create_questions.py

Questions: 100% 100/100 [00:00<00:00, 63588.60it/s]


## Test BLIP

In [8]:
%cd /content/BLIP

/content/BLIP


In [9]:
from PIL import Image
import requests
import torch
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
from models.blip_vqa import blip_vqa
from tqdm import tqdm
import pandas as pd


def load_image(image, image_size, device):
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size), interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        ]) 
    image = transform(image).unsqueeze(0).to(device)
    return image


def load_model(model_url, image_size):
    model = blip_vqa(pretrained=model_url, image_size=image_size, vit='base')
    model.eval()
    model = model.to(device)
    return model


def inference(image, question): 
    image = load_image(image, image_size, device)
    with torch.no_grad():
        output = model(image, question, train=False, inference='generate') 
    return output[0]


def test(path, df):
    answers = []
    prev_path = ""
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        image_path = path + "images/" + row['image']
        if image_path != prev_path:
            image = Image.open(image_path).convert('RGB')
        answers.append(inference(image, row['question']))
        prev_path = image_path
    df['answer'] = answers

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_size = 480
model_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model*_vqa.pth'
model = load_model(model_url, image_size)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

  0%|          | 0.00/1.35G [00:00<?, ?B/s]

load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model*_vqa.pth


### Figures

In [32]:
df_figures = pd.read_csv(path_figures + "questions.csv")
df_figures

,type,question,correct,wrong1,wrong2,image
0,Figures,How many figures?,24,23,21,figures_6_4_417148_466526_041585_724774.png
1,Figures,How many columns?,6,4,8,figures_6_4_417148_466526_041585_724774.png
2,Figures,How many rows?,4,6,2,figures_6_4_417148_466526_041585_724774.png
3,Figures,How many triangles?,6,4,5,figures_6_4_417148_466526_041585_724774.png
4,Figures,How many squares?,9,7,11,figures_6_4_417148_466526_041585_724774.png
...,...,...,...,...,...,...
1795,Figures,How many green squares?,2,0,1,figures_6_4_156668_556451_317345_333185.png
1796,Figures,How many blue squares?,6,5,8,figures_6_4_156668_556451_317345_333185.png
1797,Figures,How many red circles?,4,6,2,figures_6_4_156668_556451_317345_333185.png
1798,Figures,How many green circles?,1,3,2,figures_6_4_156668_556451_317345_333185.png


In [46]:
test(path_figures, df_figures)

100%|██████████| 1800/1800 [08:15<00:00,  3.64it/s]


In [47]:
df_figures

,type,question,correct,wrong1,wrong2,image,answer
0,Figures,How many figures?,24,23,21,figures_6_4_417148_466526_041585_724774.png,4
1,Figures,How many columns?,6,4,8,figures_6_4_417148_466526_041585_724774.png,1
2,Figures,How many rows?,4,6,2,figures_6_4_417148_466526_041585_724774.png,3
3,Figures,How many triangles?,6,4,5,figures_6_4_417148_466526_041585_724774.png,5
4,Figures,How many squares?,9,7,11,figures_6_4_417148_466526_041585_724774.png,9
...,...,...,...,...,...,...,...
1795,Figures,How many green squares?,2,0,1,figures_6_4_156668_556451_317345_333185.png,3
1796,Figures,How many blue squares?,6,5,8,figures_6_4_156668_556451_317345_333185.png,3
1797,Figures,How many red circles?,4,6,2,figures_6_4_156668_556451_317345_333185.png,3
1798,Figures,How many green circles?,1,3,2,figures_6_4_156668_556451_317345_333185.png,3


In [48]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

In [57]:
def word2int(word):
    word2int = {"one": "1", "two": "2", "three": "3", "four": "4", "red": "1", 
                "blue": "2", "yellow": "3", "left": "1"}
    if word in word2int:
        return word2int[word]
    return word

In [68]:
acc_figures = accuracy_score(list(df_figures["correct"].apply(str)), list(df_figures["answer"].apply(word2int)))
acc_figures*100

16.5

In [53]:
error_figures = mean_absolute_error(list(df_figures["correct"].apply(int)), list(df_figures["answer"].apply(word2int)))
error_figures

3.31

### Cubes

In [63]:
df_cubes = pd.read_csv(path_cubes + "questions.csv")
df_cubes

,type,question,correct,wrong1,wrong2,image
0,Cubes,How many cubes in total?,26,24,23,cubes_4_4_3_0002_0013_1133_3333.png
1,Cubes,How many visible cubes?,17,14,18,cubes_4_4_3_0002_0013_1133_3333.png
2,Cubes,How many non visible cubes?,9,7,8,cubes_4_4_3_0002_0013_1133_3333.png
3,Cubes,How many cubes in layer x 1?,2,3,4,cubes_4_4_3_0002_0013_1133_3333.png
4,Cubes,How many cubes in layer x 2?,4,3,5,cubes_4_4_3_0002_0013_1133_3333.png
...,...,...,...,...,...,...
1395,Cubes,How many cubes in layer y 3?,11,13,9,cubes_4_4_3_0023_0033_1233_2333.png
1396,Cubes,How many cubes in layer y 4?,12,15,9,cubes_4_4_3_0023_0033_1233_2333.png
1397,Cubes,How many cubes in layer z 1?,12,13,11,cubes_4_4_3_0023_0033_1233_2333.png
1398,Cubes,How many cubes in layer z 2?,11,8,12,cubes_4_4_3_0023_0033_1233_2333.png


In [64]:
test(path_cubes, df_cubes)

100%|██████████| 1400/1400 [06:24<00:00,  3.65it/s]


In [65]:
df_cubes

,type,question,correct,wrong1,wrong2,image,answer
0,Cubes,How many cubes in total?,26,24,23,cubes_4_4_3_0002_0013_1133_3333.png,12
1,Cubes,How many visible cubes?,17,14,18,cubes_4_4_3_0002_0013_1133_3333.png,6
2,Cubes,How many non visible cubes?,9,7,8,cubes_4_4_3_0002_0013_1133_3333.png,1
3,Cubes,How many cubes in layer x 1?,2,3,4,cubes_4_4_3_0002_0013_1133_3333.png,6
4,Cubes,How many cubes in layer x 2?,4,3,5,cubes_4_4_3_0002_0013_1133_3333.png,6
...,...,...,...,...,...,...,...
1395,Cubes,How many cubes in layer y 3?,11,13,9,cubes_4_4_3_0023_0033_1233_2333.png,3
1396,Cubes,How many cubes in layer y 4?,12,15,9,cubes_4_4_3_0023_0033_1233_2333.png,3
1397,Cubes,How many cubes in layer z 1?,12,13,11,cubes_4_4_3_0023_0033_1233_2333.png,3
1398,Cubes,How many cubes in layer z 2?,11,8,12,cubes_4_4_3_0023_0033_1233_2333.png,3


In [69]:
acc_cubes = accuracy_score(list(df_cubes["correct"].apply(str)), list(df_cubes["answer"].apply(word2int)))
acc_cubes*100

4.785714285714286

In [67]:
error_cubes = mean_absolute_error(list(df_cubes["correct"].apply(int)), list(df_cubes["answer"].apply(word2int)))
error_cubes

6.785714285714286

### Maze

In [19]:
df_maze = pd.read_csv(path_maze + "questions.csv")
df_maze

,type,question,correct,wrong1,wrong2,image
0,Maze,How many cells?,96,98,101,maze_0_12_8_0_2.png
1,Maze,How many colums?,12,14,11,maze_0_12_8_0_2.png
2,Maze,How many rows?,8,6,10,maze_0_12_8_0_2.png
3,Maze,Which is the exit starting from green?,blue,red,yellow,maze_0_12_8_0_2.png
4,Maze,How many cells?,96,105,91,maze_1_12_8_0_2.png
...,...,...,...,...,...,...
395,Maze,Which is the exit starting from green?,blue,red,yellow,maze_98_12_8_0_2.png
396,Maze,How many cells?,96,92,90,maze_99_12_8_0_1.png
397,Maze,How many colums?,12,13,9,maze_99_12_8_0_1.png
398,Maze,How many rows?,8,7,5,maze_99_12_8_0_1.png


In [20]:
test(path_maze, df_maze)

100%|██████████| 400/400 [02:11<00:00,  3.04it/s]


In [62]:
df_maze

,type,question,correct,wrong1,wrong2,image,answer
0,Maze,How many cells?,96,98,101,maze_0_12_8_0_2.png,8
1,Maze,How many colums?,12,14,11,maze_0_12_8_0_2.png,0
2,Maze,How many rows?,8,6,10,maze_0_12_8_0_2.png,3
3,Maze,Which is the exit starting from green?,blue,red,yellow,maze_0_12_8_0_2.png,left
4,Maze,How many cells?,96,105,91,maze_1_12_8_0_2.png,8
...,...,...,...,...,...,...,...
395,Maze,Which is the exit starting from green?,blue,red,yellow,maze_98_12_8_0_2.png,left
396,Maze,How many cells?,96,92,90,maze_99_12_8_0_1.png,8
397,Maze,How many colums?,12,13,9,maze_99_12_8_0_1.png,0
398,Maze,How many rows?,8,7,5,maze_99_12_8_0_1.png,3


In [70]:
acc_maze = accuracy_score(list(df_maze["correct"].apply(word2int)), list(df_maze["answer"].apply(word2int)))
acc_maze*100

14.249999999999998

In [60]:
error_maze = mean_absolute_error(list(df_maze["correct"].apply(word2int)), list(df_maze["answer"].apply(word2int)))
error_maze

26.39